In [ ]:
import pandas as pd 
import numpy as np
from quantopian.research.experimental import continuous_future, history


## 下準備

↓のコードをローカルで回して

1. [local_csv  自分のcsvをuploadしてResearchでつかう方法](https://www.evernote.com/Home.action#b=177d468e-bf46-408c-8f29-97655b654be5&st=p&x=upload&n=d2da4976-e9bb-42b9-bbd6-d2eeaf04ad18&t=c764b10c-336b-4f13-935d-163fe561711c)
にしたがって，データをQuantopianにUplaodする
1. /home/chidori/vxx-overnight-temp-fix/data/cboe/VX/expiry_dates.csv もアップロード

```
import pandas as pd 
vixUrl = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv'
vxstUrl = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vxstcurrent.csv'
vxvUrl = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vxvdailyprices.csv'
vxmtUrl = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vxmtdailyprices.csv'
vvixUrl = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vvixtimeseries.csv'

vix=pd.read_csv(vixUrl,  skiprows=[0], index_col = "Date", parse_dates=True)
vxst=pd.read_csv(vxstUrl,  skiprows=[0,1,2], index_col = "Date", parse_dates=True)
vxmt=pd.read_csv(vxmtUrl,  skiprows=[0,1], index_col = "Date", parse_dates=True)
vvix=pd.read_csv(vvixUrl,  skiprows=[0], index_col = "Date", parse_dates=True)

vxv=pd.read_csv(vxvUrl,  skiprows=[0,1], parse_dates=True)
vxv = vxv.set_index("Unnamed: 0")
vxv.index = pd.to_datetime(vxv.index)
vxv.index.name = "Date"
vix = vix.rename(columns={'VIX Open': 'Open',
                    'VIX High': 'High',
                    'VIX Low': 'Low',
                    'VIX Close': 'Close',
                   })

vxv.to_csv("vxv.csv")
vxst.to_csv("vxst.csv")
vxmt.to_csv("vxmt.csv")
vvix.to_csv("vvix.csv")
vix.to_csv("vix.csv")
```

## 注意

VXSTのファイルに，”*4/20/2011”みたいな，Dateに不要な文字列が入っていたりするので，入っていたら↓のLocal Csvの読み込みで失敗するので，ファイル自体を確認して下さい

In [ ]:
vix = local_csv("vix.csv", date_column="Date", use_date_column_as_index=True)
vxv = local_csv("vxv.csv", date_column="Date", use_date_column_as_index=True)
vxmt = local_csv("vxmt.csv", date_column="Date", use_date_column_as_index=True)
vvix = local_csv("vvix.csv", date_column="Date", use_date_column_as_index=True)
vxst = local_csv("vxst.csv", date_column="Date", use_date_column_as_index=True)
expiry_dates = local_csv("expiry_dates.csv")
expiry_dates = pd.to_datetime(expiry_dates["vx1"])


In [ ]:
def  rename_columns(df, prefix):
    new = [(c, prefix + "_" + c.upper()  ) for c in df.columns]
    df = df.rename(columns=dict(new))
    return df 

vix = rename_columns(vix, "VIX")
vxv = rename_columns(vxv, "VXV")
vxmt = rename_columns(vxmt, "VXMT")
# vvix = rename_columns(vvix, "VVIX")
vxst = rename_columns(vxst, "VXST")


In [ ]:
df = pd.concat([vix, vxv, vxmt, vvix, vxst], axis=1).dropna()
start = df.index[0]
stop =  df.index[-1]
xiv = get_pricing("XIV", start, stop)
vxx = get_pricing("VXX", start, stop)
spy = get_pricing("spy", start, stop)
df["XIV_CLOSE"] = xiv["price"]
df["VXX_CLOSE"] = vxx["price"]
df["SPY_CLOSE"] = spy["price"]

vx1 = continuous_future('VX')
vx2 = continuous_future('VX', offset=1)
vx1_pricing = history(vx1, fields='price', frequency='daily', start_date=start, end_date=stop)
vx2_pricing = history(vx2, fields='price', frequency='daily', start_date=start, end_date=stop)
vx1_pricing.name = "VX1"
vx2_pricing.name = "VX2"

df = df[["SPY_CLOSE","VIX_CLOSE", "VVIX","VXV_CLOSE", "VXMT_CLOSE", "VXST_CLOSE", "VXX_CLOSE", "XIV_CLOSE"]]
df = pd.concat([df, vx1_pricing, vx2_pricing], axis=1)
df = df.dropna()

In [ ]:
df["SPY_LN_RETURN"] = df["SPY_CLOSE"].apply(np.log).pct_change()
df["SPY_MSTD5"] = pd.rolling_std(df["SPY_LN_RETURN"], 5).shift(1)


df["VIX_LN_RETURN"] =  df["VIX_CLOSE"].apply(np.log).pct_change()
df["VIX_MSTD10"] = pd.rolling_std(df["VIX_LN_RETURN"], 10).shift(1)


In [ ]:
df["CONTANGO"] = df["VX2"] / df["VX1"] 



## IV 計算

IV = 前日のIV*Current Index Factor - Current Investor Fee

Current Index Factor = 現在のVX1 VX2 の取引価格 / 前日のVX1 VX2


In [ ]:
def iv(df, prev_rolldate, current_rolldate):
    # prev_rolldate = expiry_dates[113]
    # current_rolldate = expiry_dates[114]
    x = df.ix[prev_rolldate:current_rolldate]

    x["VX1 CIF"] = x["VX1"] / x["VX1"].shift(-1) 
    x["VX2 CIF"] = x["VX2"] / x["VX2"].shift(-1) 
    x["Fee"] =  0.89 / 100 ** 1/365 

    x["CntToRollDate"] = range(len(x))
    x["RemainDateRatio"] = x["CntToRollDate"] / len(x)
    x["VXX.IV"] = x["VX1"] * x["VX1 CIF"] * (1-x["RemainDateRatio"] ) +  x["VX2"] * x["VX2 CIF"] * x["RemainDateRatio"]  - x["Fee"]
    x["XIV.IV"] = x["VX2"] * x["VX2 CIF"] * (1-x["RemainDateRatio"] ) +  x["VX1"] * x["VX1 CIF"] * x["RemainDateRatio"]  - x["Fee"]    
    return x 



In [ ]:
prev_rolldate = expiry_dates[113]
current_rolldate = expiry_dates[114]
z = iv(df, prev_rolldate, current_rolldate)

In [ ]:
z[['XIV.IV','XIV_CLOSE' ]].plot(secondary_y = 'XIV_CLOSE')